<a href="https://colab.research.google.com/github/Sudheendra-RD/NLP/blob/main/SupremeCourtVerdictNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk

In [3]:
import pandas as pd
import numpy as np

In [4]:
sp = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/RNN-LSTM/justice.csv')
sp.head(2)

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights


In [5]:
sp.columns

Index(['Unnamed: 0', 'ID', 'name', 'href', 'docket', 'term', 'first_party',
       'second_party', 'facts', 'facts_len', 'majority_vote', 'minority_vote',
       'first_party_winner', 'decision_type', 'disposition', 'issue_area'],
      dtype='object')

In [6]:
del sp['Unnamed: 0']
del sp['ID']
del sp['name']
del sp['href']
del sp['docket']
del sp['term']
del sp['first_party']
del sp['second_party']
del sp['facts_len']
del sp['majority_vote']
del sp['minority_vote']
del sp['decision_type']
del sp['disposition']
del sp['issue_area']

In [7]:
sp = sp.dropna()

In [8]:
sp['first_party_winner'] = sp['first_party_winner'].replace(True, int(1))
sp['first_party_winner'] = sp['first_party_winner'].replace(False, int(0))

In [9]:
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
nltk.download('punkt')
nltk.download('wordnet')
import re

def clean_text(text):
  word = re.sub(r'[^\w\s]', ' ', str(text))
  word = re.sub(r'\d+', ' ', word)
  word = re.sub(' +', ' ', word)
  word = word.lower()
  word = word.split()
  return word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
sp['facts'] = sp['facts'].apply(clean_text)

In [13]:
len(sp)

3288

In [14]:
sp['facts'].head(2)

0    [p, in, jane, roe, a, fictional, name, used, i...
1    [p, joan, stanley, had, three, children, with,...
Name: facts, dtype: object

In [15]:
lem = WordNetLemmatizer()

In [16]:
def corp(data):
  corpus = [i for i in data if i not in set(stopwords.words('english'))]
  corpus = [lem.lemmatize(j) for j in corpus]
  return corpus

In [17]:
sp['facts'] = sp['facts'].apply(corp)

In [19]:
sp.head(2)

,facts,first_party_winner
0,"[p, jane, roe, fictional, name, used, court, d...",1
1,"[p, joan, stanley, three, child, peter, stanle...",1


In [20]:
def remove_p(p):
  wo_p = [q for q in p if q!='p']
  return str(wo_p)

In [21]:
sp['facts'] = sp['facts'].apply(remove_p)

In [22]:
sp['facts'].head(2)

0    ['jane', 'roe', 'fictional', 'name', 'used', '...
1    ['joan', 'stanley', 'three', 'child', 'peter',...
Name: facts, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(lowercase=False)
x = tf.fit_transform(sp['facts']).toarray()

In [25]:
y = pd.array(sp['first_party_winner'])
y

<PandasArray>
[1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
 ...
 0, 1, 1, 1, 1, 1, 0, 1, 0, 1]
Length: 3288, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [27]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)

In [28]:
predict = model.predict(x_test)

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predict)
acc = accuracy_score(y_test, predict)

In [30]:
cm

array([[  0, 227],
       [  0, 431]])

In [31]:
acc

0.6550151975683891